In [2]:
import pandas as pd
import sqlite3

In [16]:
query = """
SELECT *
FROM nnpes AS np
LEFT JOIN nucc_taxonomy AS nt
ON np.primary_taxonomy = nt.code
LEFT JOIN zip_cbsa AS zp
ON np.postal_code = zp.zip
WHERE entity_type_code IS NOT NULL
LIMIT 1000;
"""

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    test_sqlite = pd.read_sql(query, db)

test_sqlite

,npi,entity_type_code,provider_organization_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential_text,provider_first_line_business_practice_location_address,...,display_name,section,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,1508869488,1.0,None,HETHERINGTON,JOHN,J,DR.,None,PH.D.,177 W COTTONWOOD LN,...,Audiologist,Individual,None,NaN,None,None,NaN,NaN,NaN,NaN
1,1861495749,1.0,None,NASH,FRANK,M,None,None,CRNA,615 FAIRHURST ST,...,Certified Registered Nurse Anesthetist,Individual,80751,44540.0,STERLING,CO,1.00000,1.0,1.0,1.000000
2,1164425039,1.0,None,LEMCHEN,LAWRENCE,ARNOLD,MR.,None,R.PH.,5001 126TH AVE SE,...,Pharmacotherapy Pharmacist,Individual,98006,42660.0,BELLEVUE,WA,1.00000,1.0,1.0,1.000000
3,1982607859,1.0,None,GIST,RAYMOND,F,DR.,None,DDS,4170 LENNON RD,...,General Practice Dentistry,Individual,48507,22420.0,FLINT,MI,1.00000,1.0,1.0,1.000000
4,1528061496,1.0,None,BRODY,HOWARD,None,DR.,None,M.D.,138 SERVICE RD,...,Family Medicine Physician,Individual,48824,29620.0,EAST LANSING,MI,1.00000,1.0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1578566550,1.0,None,CIONI,JOSEPH,ALAN,DR.,None,D.M.D.,229 W SAINT LOUIS ST,...,General Practice Dentistry,Individual,62254,41180.0,LEBANON,IL,1.00000,1.0,1.0,1.000000
996,1487657466,1.0,None,WILLIAMS,TODD,EDWARD,DR.,None,D.D.S.,11325 SPRINGFIELD PIKE,...,General Practice Dentistry,Individual,45246,17140.0,CINCINNATI,OH,1.00000,1.0,1.0,1.000000
997,1831192897,1.0,None,CHMURA,LOUIS,G.,DR.,None,"D.D.S., M.S.",214 WINSTON DR,...,Orthodontics and Dentofacial Orthopedic Dentist,Individual,49068,12980.0,MARSHALL,MI,1.00000,1.0,1.0,1.000000
998,1659374619,1.0,None,MALEK,ROBERT,J.,DR.,None,D.D.S.,3539 CLIFFHANGER WAY,...,Pediatric Dentist,Individual,43701,49780.0,ZANESVILLE,OH,0.99985,1.0,1.0,0.999866


In [20]:
query = """
SELECT *
FROM nnpes AS np
INNER JOIN nucc_taxonomy AS nt
ON np.primary_taxonomy = nt.code
INNER JOIN zip_cbsa AS zc
ON np.postal_code = zc.zip
WHERE entity_type_code = 2.0
    AND state = 'TN'
    AND cbsa = 34980
    AND res_ratio >= 0.5
    AND classification LIKE '%Hospital'
    AND provider_organization_name NOT LIKE '%PAY%'
"""

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    hospitals = pd.read_sql(query, db)

hospitals

,npi,entity_type_code,provider_organization_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential_text,provider_first_line_business_practice_location_address,...,display_name,section,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,1578693545,2.0,FRANK T RUTHERFORD HOSPITAL INC,None,None,None,None,None,None,130 LEBANON HWY,...,General Acute Care Hospital,Non-Individual,37030,34980,CARTHAGE,TN,0.999715,1.0,1.0,0.999759
1,1861675647,2.0,RIVER REGION HOSPITAL,None,None,None,None,None,None,49 MUSIC SQ W,...,Psychiatric Hospital,Non-Individual,37203,34980,NASHVILLE,TN,1.000000,1.0,1.0,1.000000
2,1265445506,2.0,WILLIAMSON COUNTY HOSPITAL DISTRICT,None,None,None,None,None,None,4321 CAROTHERS PARKWAY,...,General Acute Care Hospital,Non-Individual,37067,34980,FRANKLIN,TN,1.000000,1.0,1.0,1.000000
3,1770509697,2.0,"SMITH COUNTY MEMORIAL HOSPITAL, LLC",None,None,None,None,None,None,158 HOSPITAL DR,...,General Acute Care Hospital,Non-Individual,37030,34980,CARTHAGE,TN,0.999715,1.0,1.0,0.999759
4,1609970029,2.0,"BUCKS COUNTY ONCOPLASTIC INSTITUTE, LLC",None,None,None,None,None,None,511 UNION ST,...,General Acute Care Hospital,Non-Individual,37219,34980,NASHVILLE,TN,1.000000,1.0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1184987505,2.0,"TRUSTPOINT HOSPITAL, LLC",None,None,None,None,None,None,1009 N THOMPSON LN,...,General Acute Care Hospital,Non-Individual,37129,34980,MURFREESBORO,TN,1.000000,1.0,1.0,1.000000
101,1497012959,2.0,"TRUSTPOINT HOSPITAL, LLC",None,None,None,None,None,None,1009 N THOMPSON LN,...,Psychiatric Hospital,Non-Individual,37129,34980,MURFREESBORO,TN,1.000000,1.0,1.0,1.000000
102,1821350349,2.0,"TRUSTPOINT HOSPITAL, LLC",None,None,None,None,None,None,1009 N THOMPSON LN,...,Rehabilitation Hospital,Non-Individual,37129,34980,MURFREESBORO,TN,1.000000,1.0,1.0,1.000000
103,1447762505,2.0,"NATIONAL BIRTH CENTERS, INC.",None,None,None,None,None,None,80 OAK VALLEY DR,...,General Acute Care Hospital,Non-Individual,37174,34980,SPRING HILL,TN,0.998723,1.0,1.0,0.998809


In [21]:
query = """
WITH hospitals AS (
SELECT DISTINCT npi
FROM nnpes AS np
INNER JOIN nucc_taxonomy AS nt
ON np.primary_taxonomy = nt.code
INNER JOIN zip_cbsa AS zc
ON np.postal_code = zc.zip
WHERE entity_type_code = 2.0
    AND state = 'TN'
    AND cbsa = 34980
    AND res_ratio >= 0.5
    AND classification LIKE '%Hospital'
    AND provider_organization_name NOT LIKE '%PAY%'
)
SELECT
    from_npi,
    to_npi,
    SUM(patient_count) AS patient_total,
    SUM(transaction_count) AS transaction_total
FROM hop_teaming
WHERE to_npi IN hospitals
GROUP BY from_npi, to_npi
"""

# LIMIT 1000;

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    providers_to_hospitals = pd.read_sql(query, db)

providers_to_hospitals

,from_npi,to_npi,patient_total,transaction_total
0,1003013160,1629025648,33,51
1,1003013947,1023055126,99,279
2,1003019902,1558408633,56,62
3,1003028770,1861479545,1905,2649
4,1003031261,1023055126,93,128
...,...,...,...,...
8769,1992972087,1023055126,60,98
8770,1992972087,1720032345,200,346
8771,1992985055,1942442710,122,164
8772,1992985055,1992776405,71,113


In [23]:
# this finds the total numbers for each npi
total_referrals = (
    providers_to_hospitals
    .groupby('to_npi')[['patient_total','transaction_total']]
    .sum()
    .sort_values(by='patient_total',ascending= False)
    .reset_index()
)

total_referrals

,to_npi,patient_total,transaction_total
0,1023055126,343120,587886
1,1861479545,327428,585519
2,1629025648,269061,445554
3,1164590386,166155,273259
4,1982650024,159664,256585
5,1265445506,151917,242355
6,1295780476,144364,247817
7,1447571658,122320,209280
8,1538114434,105268,163116
9,1871530832,79641,127234


In [25]:
var = tuple(total_referrals['to_npi'].tolist())

query =f"""
SELECT DISTINCT npi, provider_organization_name
FROM nnpes
WHERE npi IN {var}
"""

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    hospitals = pd.read_sql(query, db)

hospitals

,npi,provider_organization_name
0,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC."
1,1023061520,SRHS HOLDINGS LLC
2,1093769606,HTI MEMORIAL HOSPITAL CORPORATION
3,1104137215,RIVERVIEW MEDICAL CENTER LLC
4,1124065255,"HCA HEALTH SERVICES OF TENNESSEE, INC."
5,1164469227,"HCA HEALTH SERVICES OF TENNESSEE, INC."
6,1164590386,SAINT THOMAS RUTHERFORD HOSPITAL
7,1184987505,"TRUSTPOINT HOSPITAL, LLC"
8,1245541358,SMITH COUNTY MEMORIAL HOSPITAL LLC
9,1265445506,WILLIAMSON COUNTY HOSPITAL DISTRICT


In [33]:
query = '''
SELECT
    n1.npi AS from_npi,
    t1.specialization,
    n2.provider_organization_name, 
    SUM(hop.transaction_count) AS total_transactions,
    SUM(hop.patient_count) AS total_patients
FROM hop_teaming AS hop
INNER JOIN nnpes n1
ON hop.from_npi = n1.npi
INNER JOIN nnpes n2
ON hop.to_npi = n2.npi 
INNER JOIN nucc_taxonomy as t1
ON n1.primary_taxonomy = t1.code
INNER JOIN nucc_taxonomy as t2
ON n2.primary_taxonomy = t2.code
INNER JOIN zip_cbsa as zip
ON n2.postal_code = zip.zip
WHERE n1.entity_type_code = 1
    AND n2.state = 'TN'
    AND n2.entity_type_code = 2
    AND t1.specialization <> 'None'
    AND t2.classification LIKE '%Hospital'
    AND zip.cbsa = 34980
GROUP BY n2.provider_organization_name
ORDER BY total_patients DESC;
'''

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    testing_query = pd.read_sql(query, db)

testing_query


,from_npi,specialization,provider_organization_name,total_transactions,total_patients
0,1144222035,Pain Medicine,VANDERBILT UNIVERSITY MEDICAL CENTER,485507,335131
1,1538147319,"Endocrinology, Diabetes & Metabolism","HCA HEALTH SERVICES OF TENNESSEE, INC.",317409,219743
2,1225024243,Critical Care Medicine,SAINT THOMAS WEST HOSPITAL,231948,162537
3,1831191667,Medical,MAURY REGIONAL HOSPITAL,137679,92355
4,1144222035,Pain Medicine,HTI MEMORIAL HOSPITAL CORPORATION,73096,52344
5,1225024243,Critical Care Medicine,SAINT THOMAS RUTHERFORD HOSPITAL,68434,48344
6,1538147319,"Endocrinology, Diabetes & Metabolism",WILLIAMSON COUNTY HOSPITAL DISTRICT,66423,47434
7,1922084102,Emergency Medical Services,SUMNER REGIONAL MEDICAL CENTER LLC,56606,37481
8,1144222035,Pain Medicine,HENDERSONVILLE HOSPITAL CORPORATION,46960,34818
9,1316977861,Diagnostic Radiology,CENTRAL TENNESSEE HOSPITAL CORPORATION,29921,23127


In [26]:
var = tuple(total_referrals.loc[total_referrals['patient_total'] > 4000]['to_npi'].tolist())

query =f"""
SELECT *
FROM hop_teaming AS ht
INNER JOIN nnpes AS nn
ON ht.from_npi = nn.npi
LEFT JOIN nucc_taxonomy AS nc
ON nn.primary_taxonomy = nc.code
WHERE to_npi IN {var}
"""

with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    providers_to_major = pd.read_sql(query, db)

providers_to_major

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,npi,entity_type_code,provider_organization_name,provider_last_name,...,postal_code,primary_taxonomy,code,grouping,classification,specialization,definition,notes,display_name,section
0,1508869918,1861479545,36,62,36.145,32.259,1508869918,2.0,"TRI-COUNTY MEDICAL & OSTOMY SUPPLIES, INC.",None,...,37604,332B00000X,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,None,A supplier of medical equipment such as respir...,None,Durable Medical Equipment & Medical Supplies,Non-Individual
1,1508882671,1861479545,133,303,10.102,25.370,1508882671,NaN,None,None,...,None,no primary taxonomy,None,None,None,None,None,None,None,None
2,1508839622,1861479545,62,65,37.662,49.298,1508839622,1.0,None,BAKER,...,37232,207RC0000X,207RC0000X,Allopathic & Osteopathic Physicians,Internal Medicine,Cardiovascular Disease,An internist who specializes in diseases of th...,Source: American Osteopathic Board of Internal...,Cardiovascular Disease Physician,Individual
3,1508064890,1861479545,2609,2948,5.588,24.541,1508064890,1.0,None,SMITH,...,38401,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,Individual
4,1508008053,1861479545,1282,1584,18.102,42.064,1508008053,1.0,None,DINARAM,...,37232,207RI0011X,207RI0011X,Allopathic & Osteopathic Physicians,Internal Medicine,Interventional Cardiology,An area of medicine within the subspecialty of...,"Source: American Board of Medical Specialties,...",Interventional Cardiology Physician,Individual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,1417938846,1306889597,131,162,44.574,52.770,1417938846,2.0,"MACON COUNTY GENERAL HOSPITAL, INC.",None,...,37083,282NC0060X,282NC0060X,Hospitals,General Acute Care Hospital,Critical Access,Definition to come.,[7/1/2003: new],Critical Access Hospital,Non-Individual
8590,1417327446,1336119627,65,65,0.000,0.000,1417327446,1.0,None,GOMES,...,37203,367500000X,367500000X,Physician Assistants & Advanced Practice Nursi...,"Nurse Anesthetist, Certified Registered",None,(1) A licensed registered nurse with advanced ...,Sources: (1) Council on Certification of Nurse...,Certified Registered Nurse Anesthetist,Individual
8591,1417483421,1417938846,32,82,19.000,16.704,1417483421,2.0,GERIATRIC PSYCHOLOGY AFFILIATES LLC,None,...,37160,103TC0700X,103TC0700X,Behavioral Health & Social Service Providers,Psychologist,Clinical,A psychologist who provides continuing and com...,"Source: American Psychological Association, w...",Clinical Psychologist,Individual
8592,1417946203,1447571658,63,96,47.823,61.938,1417946203,1.0,None,WALLACE,...,37203,207WX0107X,207WX0107X,Allopathic & Osteopathic Physicians,Ophthalmology,Retina Specialist,An ophthalmologist who specializes in the diag...,Source: American Society of Retina Specialists...,Retina Specialist (Ophthalmology) Physician,Individual


In [27]:
(
providers_to_major
.groupby(['specialization', 'to_npi'])[['patient_count','transaction_count']]
    .sum()
    .sort_values(by='patient_count',ascending= False)
    .reset_index()
)

,specialization,to_npi,patient_count,transaction_count
0,Diagnostic Radiology,1023055126,57916,76693
1,Diagnostic Radiology,1861479545,53839,79214
2,Diagnostic Radiology,1629025648,39305,46036
3,Diagnostic Radiology,1982650024,37221,51251
4,Diagnostic Radiology,1295780476,34902,49255
...,...,...,...,...
903,Home Infusion Therapy Pharmacy,1669567897,15,59
904,Home Infusion Therapy Pharmacy,1447571658,15,56
905,Home Infusion Therapy Pharmacy,1306889597,13,50
906,Clinical Child & Adolescent,1558408633,12,154
